In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import re

#### Adding SECID to firmquarter

In [3]:
# Adding SECID to firmquarter

firmquarter = pd.read_csv("firmquarter.csv")
secid_df = pd.read_csv("secid.csv")

# Same CUSIP for both files
firmquarter['cusip8'] = firmquarter['cusip'].astype(str).str.strip().str[:8]
secid_df['cusip8'] = secid_df['cusip'].astype(str).str.strip().str[:8]

# Parsing dates
secid_df['effect_date'] = pd.to_datetime(secid_df['effect_date'], errors='coerce')
firmquarter['date_earningscall'] = pd.to_datetime(firmquarter['date_earningscall'], errors='coerce')
secid_clean = secid_df[['secid', 'cusip8', 'effect_date']].dropna()
merged = firmquarter.merge(secid_clean, on='cusip8', how='left') #Merge many-to-many by cusip8

# Filtering only records with effect_date <= date_earningscall
merged = merged[merged['effect_date'] <= merged['date_earningscall']]
merged = merged.sort_values(['gvkey', 'date_earningscall', 'effect_date'])
merged = merged.drop_duplicates(subset=['gvkey', 'date_earningscall'], keep='last') # last valid earning call
firmquarter_with_secid = merged.copy()


firmquarter_with_secid.to_csv("firmquarter_with_secid.csv", index=False)


/tmp/ipykernel_85247/1284085019.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  secid_df['effect_date'] = pd.to_datetime(secid_df['effect_date'], errors='coerce')
/tmp/ipykernel_85247/1284085019.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  firmquarter['date_earningscall'] = pd.to_datetime(firmquarter['date_earningscall'], errors='coerce')


In [4]:
# Converting to .txt (to download from Compustat)

prisk_secid = pd.read_csv("firmquarter_with_secid.csv")

# Unique SECID, converted to a string no decimals
secid_list = (
    prisk_secid['secid']
    .dropna()
    .drop_duplicates()
    .astype(str)
    .str.replace(r'\.0$', '', regex=True)
    .str.strip()
)

# to .txt
secid_list.to_csv("secid_hassan.txt", index=False, header=False)
with open("secid_hassan.txt", "r") as f:
    lines = f.readlines()

print(f"Total secid: {len(lines)}")
print(lines[:5])


Total secid: 6533
['189845\n', '102510\n', '109024\n', '214847\n', '5007\n']


In [5]:
# Joining firmquarter secid 2002-2016 with 2017-2025

firmquarter_old = pd.read_csv("firmquarter_with_secid.csv")
scores_new = pd.read_csv("scores_2017-25quarter.csv")

# Keys consistency
for col in ['gvkey', 'secid']:
    firmquarter_old[col] = pd.to_numeric(firmquarter_old[col], errors='coerce')
    scores_new[col] = pd.to_numeric(scores_new[col], errors='coerce')

firmquarter_old['quarter'] = firmquarter_old['quarter'].astype(str).str.upper()
scores_new['quarter'] = scores_new['quarter'].astype(str).str.upper()
firmquarter_full = pd.concat([firmquarter_old, scores_new], ignore_index=True)

firmquarter_full.to_csv("firmquarter_with_secid_full.csv", index=False)
print(f"File saved with {len(firmquarter_full)} rows and {len(firmquarter_full.columns)} columns")

File saved with 185616 rows and 50 columns


#### Implied Volatility Construction

In [6]:
# Merging Firmquarter_with_SECID with existing SECIDs in the Impvol file

prisk_secid = pd.read_csv("firmquarter_with_secid_full.csv")
impvol1 = pd.read_csv("01-01-02 to 01-01-17.csv")
impvol2 = pd.read_csv("01-01-17 to 31-08-22.csv")
impvol3 = pd.read_csv("31-08-22 to 31-08-23.csv")

/tmp/ipykernel_85247/3527273790.py:3: DtypeWarning: Columns (41,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  prisk_secid = pd.read_csv("firmquarter_with_secid_full.csv")
/tmp/ipykernel_85247/3527273790.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  impvol1 = pd.read_csv("01-01-02 to 01-01-17.csv")


In [8]:
# Merge and SECID Filtering

prisk_secid = pd.read_csv("firmquarter_with_secid_full.csv")
secids_validos = set(prisk_secid['secid'].dropna().unique())

# -------------------------------
# Merge and filtering
# -------------------------------
def merge_y_filtrar_impvol(files, output_file, secids_validos, chunksize=500_000):
    total_guardadas = 0
    
    for f in files:
        for i, chunk in enumerate(pd.read_csv(f, chunksize=chunksize)):
            filtrado = chunk[chunk['secid'].isin(secids_validos)]
            
            mode = 'a' if total_guardadas > 0 else 'w'
            header = total_guardadas == 0
            
            filtrado.to_csv(output_file, mode=mode, header=header, index=False)
            total_guardadas += len(filtrado)
            
            print(f"[{f}] Chunk {i+1}: {len(filtrado)} saved rows (Total: {total_guardadas})")
    
    print(f"Final file saved: {output_file}")

# Execute
merge_y_filtrar_impvol(
    ["01-01-02 to 01-01-17.csv", "01-01-17 to 31-08-22.csv", "31-08-22 to 31-08-23.csv"],
    "impvol_filtered.csv",
    secids_validos
)


/tmp/ipykernel_85247/1336657472.py:3: DtypeWarning: Columns (41,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  prisk_secid = pd.read_csv("firmquarter_with_secid_full.csv")


[01-01-02 to 01-01-17.csv] Chunk 1: 170628 saved rows (Total: 170628)
[01-01-02 to 01-01-17.csv] Chunk 2: 179246 saved rows (Total: 349874)
[01-01-02 to 01-01-17.csv] Chunk 3: 154450 saved rows (Total: 504324)
[01-01-02 to 01-01-17.csv] Chunk 4: 121446 saved rows (Total: 625770)
[01-01-02 to 01-01-17.csv] Chunk 5: 387458 saved rows (Total: 1013228)
[01-01-02 to 01-01-17.csv] Chunk 6: 433032 saved rows (Total: 1446260)
[01-01-02 to 01-01-17.csv] Chunk 7: 367138 saved rows (Total: 1813398)
[01-01-02 to 01-01-17.csv] Chunk 8: 403010 saved rows (Total: 2216408)
[01-01-02 to 01-01-17.csv] Chunk 9: 374774 saved rows (Total: 2591182)
[01-01-02 to 01-01-17.csv] Chunk 10: 374826 saved rows (Total: 2966008)
[01-01-02 to 01-01-17.csv] Chunk 11: 436784 saved rows (Total: 3402792)
[01-01-02 to 01-01-17.csv] Chunk 12: 367332 saved rows (Total: 3770124)
[01-01-02 to 01-01-17.csv] Chunk 13: 412162 saved rows (Total: 4182286)
[01-01-02 to 01-01-17.csv] Chunk 14: 370734 saved rows (Total: 4553020)


/tmp/ipykernel_85247/1336657472.py:13: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(f, chunksize=chunksize)):


[01-01-02 to 01-01-17.csv] Chunk 15: 260384 saved rows (Total: 4813404)


/tmp/ipykernel_85247/1336657472.py:13: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(f, chunksize=chunksize)):


[01-01-02 to 01-01-17.csv] Chunk 16: 281700 saved rows (Total: 5095104)


/tmp/ipykernel_85247/1336657472.py:13: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(f, chunksize=chunksize)):


[01-01-02 to 01-01-17.csv] Chunk 17: 423868 saved rows (Total: 5518972)
[01-01-02 to 01-01-17.csv] Chunk 18: 426178 saved rows (Total: 5945150)
[01-01-02 to 01-01-17.csv] Chunk 19: 396856 saved rows (Total: 6342006)
[01-01-02 to 01-01-17.csv] Chunk 20: 363318 saved rows (Total: 6705324)
[01-01-02 to 01-01-17.csv] Chunk 21: 404330 saved rows (Total: 7109654)
[01-01-02 to 01-01-17.csv] Chunk 22: 368514 saved rows (Total: 7478168)
[01-01-02 to 01-01-17.csv] Chunk 23: 415098 saved rows (Total: 7893266)
[01-01-02 to 01-01-17.csv] Chunk 24: 396326 saved rows (Total: 8289592)
[01-01-02 to 01-01-17.csv] Chunk 25: 355852 saved rows (Total: 8645444)
[01-01-02 to 01-01-17.csv] Chunk 26: 389240 saved rows (Total: 9034684)
[01-01-02 to 01-01-17.csv] Chunk 27: 401664 saved rows (Total: 9436348)
[01-01-02 to 01-01-17.csv] Chunk 28: 367560 saved rows (Total: 9803908)
[01-01-02 to 01-01-17.csv] Chunk 29: 400972 saved rows (Total: 10204880)
[01-01-02 to 01-01-17.csv] Chunk 30: 381846 saved rows (Total: 

/tmp/ipykernel_85247/1336657472.py:13: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(f, chunksize=chunksize)):


[01-01-17 to 31-08-22.csv] Chunk 5: 306458 saved rows (Total: 17348194)
[01-01-17 to 31-08-22.csv] Chunk 6: 438194 saved rows (Total: 17786388)
[01-01-17 to 31-08-22.csv] Chunk 7: 392800 saved rows (Total: 18179188)
[01-01-17 to 31-08-22.csv] Chunk 8: 410990 saved rows (Total: 18590178)
[01-01-17 to 31-08-22.csv] Chunk 9: 405960 saved rows (Total: 18996138)
[01-01-17 to 31-08-22.csv] Chunk 10: 334182 saved rows (Total: 19330320)
[01-01-17 to 31-08-22.csv] Chunk 11: 274758 saved rows (Total: 19605078)
[01-01-17 to 31-08-22.csv] Chunk 12: 300714 saved rows (Total: 19905792)
[01-01-17 to 31-08-22.csv] Chunk 13: 217198 saved rows (Total: 20122990)
[01-01-17 to 31-08-22.csv] Chunk 14: 195684 saved rows (Total: 20318674)
[01-01-17 to 31-08-22.csv] Chunk 15: 254768 saved rows (Total: 20573442)
[01-01-17 to 31-08-22.csv] Chunk 16: 146478 saved rows (Total: 20719920)
[01-01-17 to 31-08-22.csv] Chunk 17: 267370 saved rows (Total: 20987290)
[01-01-17 to 31-08-22.csv] Chunk 18: 258464 saved rows (

In [9]:
df = pd.read_csv("impvol_filtered.csv", parse_dates=['date'])
df['year'] = df['date'].dt.year
obs_por_ano = df.groupby('year').size().reset_index(name='n_obs')
obs_por_ano = obs_por_ano.sort_values('year')

print(obs_por_ano)

/tmp/ipykernel_85247/3544383503.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("impvol_filtered.csv", parse_dates=['date'])


    year    n_obs
0   2002   674782
1   2003   658570
2   2004   717404
3   2005   800472
4   2006   867528
5   2007   960714
6   2008  1026904
7   2009  1037392
8   2010  1073346
9   2011  1162414
10  2012  1209564
11  2013  1283344
12  2014  1359702
13  2015  1400464
14  2016  1428964
15  2017  1386390
16  2018  1385138
17  2019  1368820
18  2020  1361470
19  2021  1477424
20  2022  1538248
21  2023   979802


#### Implied Volatility Calculation

In [11]:
# -------------------------------
# Function for working with ImpVol in chunks
# -------------------------------
def colapsar_impvol_hassan(input_file, output_file, chunksize=500_000):
    temp_results = []
    
    for i, chunk in enumerate(pd.read_csv(input_file, chunksize=chunksize, parse_dates=['date'])):
        chunk = chunk[chunk['days'] == 91].copy() # 90-day at-the-money options
        chunk['quarter'] = chunk['date'].dt.to_period('Q').astype(str).str.upper()
        agg = chunk.groupby(['secid', 'quarter'])['impl_volatility'].mean().reset_index() # Mean by Q
        agg.rename(columns={'impl_volatility': 'impvol_mean'}, inplace=True)
        
        temp_results.append(agg)
        print(f"Chunk {i+1}: {len(agg)} rows")
    
    # Concatenate
    final_df = pd.concat(temp_results, ignore_index=True)
    final_df = final_df.groupby(['secid', 'quarter'])['impvol_mean'].mean().reset_index() # Group duplicated
    
    final_df.to_csv(output_file, index=False)
    print(f"File Saved: {output_file}")
    return final_df

# -------------------------------
# Hassan variable specifications calculation
# -------------------------------
impvol_collapsed = colapsar_impvol_hassan("impvol_filtered.csv", "impvol_collapsed_raw.csv")

p01 = impvol_collapsed['impvol_mean'].quantile(0.01)
p99 = impvol_collapsed['impvol_mean'].quantile(0.99)
impvol_collapsed['impvol_winz'] = impvol_collapsed['impvol_mean'].clip(lower=p01, upper=p99) # Winsorization same as RV

mean_ = impvol_collapsed['impvol_winz'].mean()
std_ = impvol_collapsed['impvol_winz'].std()
impvol_collapsed['impvol_std'] = (impvol_collapsed['impvol_winz'] - mean_) / std_ # Standardization


impvol_collapsed[['secid', 'quarter', 'impvol_std']].to_csv("impvol_hassan_style.csv", index=False)
print("File saved as 'impvol_hassan_style.csv'")

Chunk 1: 4048 rows
Chunk 2: 4047 rows
Chunk 3: 4045 rows
Chunk 4: 4020 rows
Chunk 5: 4042 rows
Chunk 6: 4027 rows
Chunk 7: 4029 rows
Chunk 8: 4034 rows
Chunk 9: 4030 rows
Chunk 10: 4023 rows


/tmp/ipykernel_85247/2621002922.py:7: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(input_file, chunksize=chunksize, parse_dates=['date'])):


Chunk 11: 4038 rows
Chunk 12: 4029 rows


/tmp/ipykernel_85247/2621002922.py:7: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(input_file, chunksize=chunksize, parse_dates=['date'])):


Chunk 13: 4053 rows
Chunk 14: 4034 rows
Chunk 15: 4026 rows
Chunk 16: 4039 rows
Chunk 17: 4027 rows
Chunk 18: 4024 rows
Chunk 19: 4042 rows
Chunk 20: 4046 rows
Chunk 21: 4041 rows
Chunk 22: 4035 rows
Chunk 23: 4065 rows
Chunk 24: 4078 rows
Chunk 25: 4107 rows
Chunk 26: 4087 rows
Chunk 27: 4089 rows
Chunk 28: 4086 rows
Chunk 29: 4100 rows
Chunk 30: 4111 rows
Chunk 31: 4167 rows
Chunk 32: 4151 rows
Chunk 33: 4056 rows
Chunk 34: 4055 rows
Chunk 35: 4059 rows
Chunk 36: 4054 rows
Chunk 37: 4054 rows
Chunk 38: 4061 rows
Chunk 39: 4061 rows
Chunk 40: 4067 rows
Chunk 41: 4080 rows
Chunk 42: 4078 rows
Chunk 43: 4090 rows
Chunk 44: 4080 rows
Chunk 45: 4096 rows
Chunk 46: 4184 rows
Chunk 47: 4322 rows
Chunk 48: 4895 rows
Chunk 49: 5001 rows
Chunk 50: 5050 rows
Chunk 51: 1589 rows
File Saved: impvol_collapsed_raw.csv
File saved as 'impvol_hassan_style.csv'


In [12]:
# Joining Firmquarter with Secid (To then run regressions)

firmquarter = pd.read_csv("firmquarter_with_secid_full.csv")
impvol = pd.read_csv("impvol_hassan_style.csv")
capinv = pd.read_csv("capinv.csv")  # ATQ (Assets)

firmquarter['secid'] = pd.to_numeric(firmquarter['secid'], errors='coerce') # Formating 
impvol['secid'] = pd.to_numeric(impvol['secid'], errors='coerce')

firmquarter['quarter'] = firmquarter['quarter'].astype(str).str.strip().str.upper()
impvol['quarter'] = impvol['quarter'].astype(str).str.strip().str.upper()

# -------------------------------
# Log_assets calculation
# -------------------------------
capinv['datadate'] = pd.to_datetime(capinv['datadate'], errors='coerce')
capinv = capinv.dropna(subset=['datadate', 'atq'])
capinv = capinv[capinv['atq'] > 0].copy()

capinv['quarter'] = capinv['datadate'].dt.to_period('Q').astype(str).str.strip().str.upper()
capinv['log_assets'] = np.log(capinv['atq'])
capinv['gvkey'] = pd.to_numeric(capinv['gvkey'], errors='coerce')

capinv_subset = capinv[['gvkey', 'quarter', 'log_assets']].drop_duplicates()

panel_final = firmquarter.merge( # Merging Firmquarter (PRisk) + Impvol
    impvol[['secid', 'quarter', 'impvol_std']],
    on=['secid', 'quarter'],
    how='left'
)

panel_final['gvkey'] = pd.to_numeric(panel_final['gvkey'], errors='coerce') # Merging with Log(Assets)
panel_final = panel_final.merge(
    capinv_subset,
    on=['gvkey', 'quarter'],
    how='left' 
)

# Save file
panel_final.to_csv("panel_final_impvol_with_assets.csv", index=False)
print(f"Final panel: {len(panel_final)}")
print(f"Impvol Obs: {panel_final['impvol_std'].notna().sum()}")
print(f"Log Assets Obs: {panel_final['log_assets'].notna().sum()}")


/tmp/ipykernel_85247/1540914549.py:3: DtypeWarning: Columns (41,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  firmquarter = pd.read_csv("firmquarter_with_secid_full.csv")


Final panel: 185617 filas
Impvol Obs: 147291
Log Assets Obs: 183620


##### OLS + FE Regressions (Hassan)

In [7]:
# Uncomment Model 5 (Sometimes it gets stuck)

panel = pd.read_csv("panel_final_impvol_with_assets.csv")

# Cleaning NAs
panel.replace(['na', 'NA', 'NaN', 'nan', '', 'NONE', 'None'], np.nan, inplace=True)

cols_to_numeric = ['PRisk', 'impvol_std', 'log_assets', 'SIC']
for col in cols_to_numeric:
    panel[col] = pd.to_numeric(panel[col], errors='coerce')

# Filtering for 2002–2016
panel['quarter'] = panel['quarter'].astype(str).str.strip().str.upper()
panel['year'] = panel['quarter'].str[:4].astype(int)
panel = panel[(panel['year'] >= 2002) & (panel['year'] <= 2016)].copy()

#Variable creation for OLS Specification
panel['sic2'] = panel['SIC'].floordiv(100)
panel = panel.dropna(subset=['sic2'])
panel['sic2'] = panel['sic2'].astype(int).astype(str)

panel['gvkey'] = panel['gvkey'].astype(str)
panel['PRisk_std'] = (panel['PRisk'] - panel['PRisk'].mean()) / panel['PRisk'].std()

subset = panel.dropna(subset=[
    'impvol_std', 'PRisk_std', 'log_assets', 'quarter', 'gvkey'
]).copy()

# -------------------------------
# Regressions – Panel A
# -------------------------------

# Col 1: Solo PRisk
model1 = smf.ols("impvol_std ~ PRisk_std", data=subset).fit(
    cov_type='cluster', cov_kwds={'groups': subset['gvkey']})

# Col 2: + log_assets
model2 = smf.ols("impvol_std ~ PRisk_std + log_assets", data=subset).fit(
    cov_type='cluster', cov_kwds={'groups': subset['gvkey']})

# Col 3: + quarter FE
model3 = smf.ols("impvol_std ~ PRisk_std + C(quarter)", data=subset).fit(
    cov_type='cluster', cov_kwds={'groups': subset['gvkey']})

# Col 4: + quarter FE + sector FE
model4 = smf.ols("impvol_std ~ PRisk_std + log_assets + C(quarter) + C(sic2)", data=subset).fit(
    cov_type='cluster', cov_kwds={'groups': subset['gvkey']})

# Col 5: + quarter FE + firm FE
#model5 = smf.ols("impvol_std ~ PRisk_std + log_assets + C(quarter) + C(gvkey)", data=subset).fit(
    #cov_type='cluster', cov_kwds={'groups': subset['gvkey']})

# -------------------------------
# Results table
# -------------------------------
models = [model1, model2, model3, model4] #model5]
labels = ['(1)', '(2)', '(3)', '(4)'] #'(5)']

def format_coef(model, var='PRisk_std'):
    coef = model.params[var]
    se = model.bse[var]
    pval = model.pvalues[var]

    stars = ''
    if pval < 0.01:
        stars = '***'
    elif pval < 0.05:
        stars = '**'
    elif pval < 0.1:
        stars = '*'

    coef_str = f"{coef:.3f}{stars}"
    se_str = f"({se:.3f})"
    return coef_str, se_str

table_data = {'PRisk_std': [], '': [], 'N': [], 'Adj. R²': []}

for m in models:
    coef, se = format_coef(m)
    table_data['PRisk_std'].append(coef)
    table_data[''].append(se)
    table_data['N'].append(int(m.nobs))
    table_data['Adj. R²'].append(f"{m.rsquared_adj:.3f}")

table_df = pd.DataFrame(table_data, index=labels).T

print("\nPanel A (Table IV): impvol_std ~ PRisk_std (2002–2016)")
print(table_df.to_markdown())


/tmp/ipykernel_87214/1367912347.py:3: DtypeWarning: Columns (41,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  panel = pd.read_csv("panel_final_impvol_with_assets.csv")



Panel A (Table IV): impvol_std ~ PRisk_std (2002–2016)
|           | (1)     | (2)      | (3)     | (4)      |
|:----------|:--------|:---------|:--------|:---------|
| PRisk_std | 0.016** | 0.030*** | 0.004   | 0.015*** |
|           | (0.007) | (0.006)  | (0.007) | (0.006)  |
| N         | 87562   | 87562    | 87562   | 87562    |
| Adj. R²   | 0.001   | 0.200    | 0.172   | 0.412    |


##### Adding Lobby

In [8]:
# Lobbying
lobby = pd.read_csv("Lobby_final_gvkeys_fuzzy.csv")
lobby = lobby[lobby["gvkey"].notna()]
lobby["gvkey"] = lobby["gvkey"].astype(str).str.strip()
lobby["gvkey"] = lobby["gvkey"].astype(int).astype(str).str.zfill(6)

# Panel
panel = pd.read_csv("panel_final_impvol_with_assets.csv")
panel = panel[panel["gvkey"].notna()]
panel["gvkey"] = panel["gvkey"].astype(float).astype(int).astype(str).str.zfill(6)

# Intersection
firmas_comunes = set(lobby["gvkey"]).intersection(set(panel["gvkey"]))
print("Firms with Lobby in final panel:", len(firmas_comunes))


Firms with Lobby in final panel: 1826


/tmp/ipykernel_87214/1941310536.py:8: DtypeWarning: Columns (41,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  panel = pd.read_csv("panel_final_impvol_with_assets.csv")


In [9]:
# Final Merge

panel = pd.read_csv("panel_final_impvol_with_assets.csv", low_memory=False)
panel["year"] = panel["quarter"].str.extract(r"(\d{4})").astype("Int64")
panel["quarter_num"] = panel["quarter"].str.extract(r"Q([1-4])").astype("Int64")
panel = panel.drop(columns=["quarter"])
panel = panel.rename(columns={"quarter_num": "quarter"})

# Checkin result
print(panel[["year", "quarter"]].dropna().drop_duplicates().tail())


        year  quarter
184328  2023        3
184331  2024        3
184338  2022        2
184359  2025        2
184360  2024        2


In [10]:
# Loading Lobbying file

lobby = pd.read_csv("Lobby_final_gvkeys_fuzzy.csv")

# Cleaning Lobby panel 

lobby["gvkey"] = pd.to_numeric(lobby["gvkey"], errors="coerce")
lobby = lobby[lobby["gvkey"].notna()]
lobby["gvkey"] = lobby["gvkey"].astype(int).astype(str).str.zfill(6)

lobby["year"] = pd.to_numeric(lobby["year"], errors="coerce").astype("Int64")
lobby["quarter"] = pd.to_numeric(lobby["quarter"], errors="coerce").astype("Int64")

# Cleaning main panel

panel = pd.read_csv("panel_final_impvol_with_assets.csv", low_memory=False)

panel["year"] = panel["quarter"].str.extract(r"(\d{4})").astype("Int64")
panel["quarter"] = panel["quarter"].str.extract(r"Q([1-4])").astype("Int64")

panel["gvkey"] = pd.to_numeric(panel["gvkey"], errors="coerce")
panel = panel[panel["gvkey"].notna()]
panel["gvkey"] = panel["gvkey"].astype(int).astype(str).str.zfill(6)

panel_merged = panel.merge(lobby, on=["gvkey", "year", "quarter"], how="left") # Merge panel + lobby

# Lobbying variables for Regressions
panel_merged["lobby_usd"] = pd.to_numeric(panel_merged["lobby_usd"], errors="coerce")
panel_merged["log_lobby_usd"] = np.log1p(panel_merged["lobby_usd"])
panel_merged["has_lobbying"] = panel_merged["lobby_usd"].notna().astype(int)

# Results

panel_merged.to_csv("panel_with_lobbying_impvol.csv", index=False)
print("Obs wiht lobbying:", panel_merged["has_lobbying"].sum())
print("Firms with lobbying:", panel_merged.loc[panel_merged["has_lobbying"]==1, "gvkey"].nunique())


Obs wiht lobbying: 29055
Firms with lobbying: 1301
